# Setup

In [1]:
# Hardware-aware device selection
import torch

def get_device() -> str:
    """
    Returns the optimal device string for running PyTorch models, prioritizing CUDA (NVIDIA GPUs),
    Apple Silicon MPS (Metal), and falling back to CPU. Optionally checks for AMD ROCm/HIP.

    Returns
    ----------
    - device: str
        Name of the device to use: "cuda", "mps", "cpu", or "hip".

    Notes
    ----------
    - On NVIDIA GPUs, "cuda" is used for hardware acceleration.
    - On Apple Silicon, "mps" uses Metal Performance Shaders, providing hardware acceleration.
    - On AMD, "hip" (ROCm) may be available, but this is uncommon.
    - If no GPU is available, it defaults to "cpu".
    Raises
    ----------
    - None
    """
    # Check for CUDA (NVIDIA GPUs)
    if torch.cuda.is_available():
        return "cuda"
    # or MPS (Apple Silicon)
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        return "mps"
    # or ROCm (AMD)
    if hasattr(torch.version, "hip") and torch.version.hip is not None:
        return "hip"
    # Fallback to CPU
    return "cpu"

device = get_device()
print(f"Using device: {device}")


Using device: mps


# Building the emotionRegressor

This class loads both transformers, runs inference, and outputs the 35d mood vector for a single input string. Handles device placement and memory cleanup.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

class EmotionRegressor:
    """
    Loads and manages transformer model for emotional analysis, producing a 28-dimensional mood vector for a given text input.
    
    Models:
    ----------
    - SamLowe/roberta-base-go_emotions: Predicts confidence scores for 28 discrete emotions (GoEmotions).

    Parameters
    ----------
    - individual_emotion_model_name: str
        HuggingFace model name for the 28d emotion regressor (default: "SamLowe/roberta-base-go_emotions").
        - device: str or None
        Device to use for inference ("cuda", "mps", "cpu", etc). If None, selects best available.

    Methods
    -------
    - predict(text: str) -> np.ndarray
        Returns a 28d mood vector for a given text.
    - cleanup() -> None
        Frees model memory for safe re-initialization.
    """
    def __init__(self,
                 individual_emotion_model_name: str = "SamLowe/roberta-base-go_emotions",
                 device: str = None):
        """
        Initialize the EmotionRegressor and load both models on the appropriate device.

        Parameters
        ----------
        - individual_emotion_model_name: str
            Name or path for the 28-emotion model.

        - device: str or None
            Device for inference.

        Returns
        -------
        - None

        Raises
        -------
        - RuntimeError: if model loading fails or device is invalid.
        """
        # Load SamLowe model for individual emotions
        self.device = device or get_device()
        self.individual_emotion_tokenizer = AutoTokenizer.from_pretrained(individual_emotion_model_name)
        self.individual_emotion_model = AutoModelForSequenceClassification.from_pretrained(individual_emotion_model_name).to(self.device)
        self.individual_emotion_model.eval()

    def predict(self, text: str) -> np.ndarray:
        """
        Generates a 28-dimensional mood vector for the given input text.

        Parameters
        ----------
        - text: str
            The text to analyze for emotion content.

        Returns
        -------
        - mood_vector: np.ndarray, shape (28,)
            Concatenated vector: [28d GoEmotions scores]

        Raises
        -------
        - ValueError: if the text input is empty or not a string.
        - RuntimeError: if inference fails due to hardware or model issues.
        """
        # Validate input
        if not isinstance(text, str) or not text.strip():
            raise ValueError("Input text must be a non-empty string.")
        
        # Lowe model output (28d)
        inputs = self.individual_emotion_tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(self.device)
        with torch.no_grad():
            individual_emotion_logits = self.individual_emotion_model(**inputs).logits
            individual_emotion_probs = torch.sigmoid(individual_emotion_logits).cpu().numpy().flatten()  # (28,)
            
        # Combine into 35d vector
        mood_vector = np.concatenate([individual_emotion_probs])
        return mood_vector

    def cleanup(self) -> None:
        """
        Releases GPU/CPU memory by deleting model weights.

        Returns
        -------
        - None
        """
        # Delete models to free memory
        del self.individual_emotion_model
        import gc
        # Run garbage collection to free up memory
        gc.collect()
        # Clear CUDA cache if using GPU
        if self.device == "cuda":
            import torch
            torch.cuda.empty_cache()


# Demo of 28-dimensional mood_vector

## Hard-coded example

In [3]:
# Initialize and demo
emotion_regressor = EmotionRegressor(device=device)
sample_text = "ugh. like, im just so OVER everything right now. i have like, three midterms next week and a huge group project that my group members rNT even helping with. i stayed up till 4 am trying to finISH something for it but like, im just SO tired all the time. and my brain feels like mush. 😵‍💫 i just cant focus on anythinG. i keep staring at my textbook but the words just blur together."
mood_vector = emotion_regressor.predict(sample_text)
print("28d mood_vector:", mood_vector)
print("Shape:", mood_vector.shape)
emotion_regressor.cleanup()


28d mood_vector: [2.8023950e-03 5.8483742e-03 2.6501410e-02 6.0792714e-01 1.7827628e-02
 1.3144811e-03 4.6299598e-03 1.8998149e-03 7.1649766e-03 4.0156025e-01
 7.6483473e-02 2.3426151e-02 1.1409246e-02 2.5509598e-03 1.2240275e-03
 5.3117541e-04 1.2875771e-03 2.6198630e-03 1.4335144e-03 4.5935083e-03
 3.8116251e-03 9.4346644e-04 4.1253220e-02 1.7023967e-03 1.7657561e-03
 3.4906741e-02 3.8284496e-03 1.1978948e-01]
Shape: (28,)


## Tinkering with output display format

In [4]:
import pandas as pd

# These should be the order of the GoEmotions and Ekman categories used by your models
GOEMOTIONS_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment',
    'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness',
    'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]

def pretty_print_mood_vector(mood_vector):
    """
    Prints the 28d mood vector in a readable format using pandas DataFrame.
    """
    if len(mood_vector) != 28:
        raise ValueError("Mood vector must be length 28")
    data = {**{f"{e}": v for e, v in zip(GOEMOTIONS_LABELS, mood_vector[:28])}}
    df = pd.DataFrame([data])
    print(df.T.rename(columns={0: 'confidence'}))  # .T to flip to (label, value) per row

# Usage
pretty_print_mood_vector(mood_vector)




                confidence
admiration        0.002802
amusement         0.005848
anger             0.026501
annoyance         0.607927
approval          0.017828
caring            0.001314
confusion         0.004630
curiosity         0.001900
desire            0.007165
disappointment    0.401560
disapproval       0.076483
disgust           0.023426
embarrassment     0.011409
excitement        0.002551
fear              0.001224
gratitude         0.000531
grief             0.001288
joy               0.002620
love              0.001434
nervousness       0.004594
optimism          0.003812
pride             0.000943
realization       0.041253
relief            0.001702
remorse           0.001766
sadness           0.034907
surprise          0.003828
neutral           0.119789


In [5]:
emotion_regressor = EmotionRegressor(device=device)


# live use (single-input)

In [6]:
# user_input = input("Enter text: ")
# mood_vector = emotion_regressor.predict(user_input)


# print(user_input)
# pretty_print_mood_vector(mood_vector)

# print(mood_vector)
# # list(mood_vector)
# len(mood_vector)

# Mustafar Demo

In [7]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Absolute path, since the notebook lives elsewhere
ROTS_PATH = "/Users/dansantoro/Desktop/project_therapy/datasets/rots.csv"

# Load dataset
rots_df = pd.read_csv(ROTS_PATH)

# Filter Anakin lines, preserve order
anakin_df = rots_df[rots_df["SPEAKER"] == "Anakin"].reset_index(drop=True)

# Collect utterances + emotion vectors
rows = []

for utterance in anakin_df["UTTERANCE"]:
    emotions = emotion_regressor.predict(utterance)
    rows.append([utterance] + list(emotions))

# Build TrendTracker-style dataframe
anakin_trendtracker = pd.DataFrame(
    rows,
    columns=[
        "utterance",
        "admiration", "amusement", "anger", "annoyance", "approval", "caring",
        "confusion", "curiosity", "desire", "disappointment", "disapproval",
        "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief",
        "joy", "love", "nervousness", "optimism", "pride", "realization",
        "relief", "remorse", "sadness", "surprise", "neutral"
    ]
)

display(anakin_trendtracker)

,utterance,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,I saw your ship. What are you doing out here?,0.005758,0.001704,0.001566,0.004761,0.011941,0.001409,0.167364,0.633605,0.001742,0.002052,0.001808,0.001126,0.000838,0.009665,0.002376,0.000845,0.000401,0.001910,0.004675,0.001210,0.003146,0.000116,0.010838,0.000296,0.000513,0.001328,0.031930,0.292631
1,What things?,0.001477,0.001454,0.002698,0.008272,0.006886,0.000977,0.140438,0.290607,0.001064,0.001476,0.002237,0.001187,0.000580,0.002266,0.000871,0.000662,0.000259,0.000835,0.001302,0.000350,0.001855,0.000071,0.004315,0.000146,0.000254,0.000837,0.006319,0.767112
2,Obi-Wan is trying to turn you against me.,0.001119,0.003742,0.015722,0.039175,0.006835,0.002075,0.001453,0.001190,0.001996,0.003991,0.003704,0.003925,0.001030,0.001299,0.001714,0.000509,0.000368,0.001455,0.000657,0.000375,0.003276,0.000299,0.004162,0.000275,0.000324,0.002209,0.000864,0.941642
3,Us?,0.001479,0.001644,0.002167,0.007315,0.008448,0.000899,0.192334,0.243566,0.000816,0.001354,0.002561,0.001216,0.000657,0.002191,0.000993,0.000703,0.000252,0.000921,0.001332,0.000376,0.001850,0.000080,0.005306,0.000158,0.000274,0.000803,0.006002,0.763730
4,"Love won't save you, Padme. Only my new powers can do that.",0.021315,0.000685,0.003990,0.005125,0.260631,0.535207,0.001789,0.002508,0.026089,0.002198,0.009200,0.001145,0.000407,0.004558,0.002016,0.007333,0.001137,0.005327,0.460228,0.001299,0.117254,0.001937,0.006563,0.003311,0.002821,0.002524,0.000578,0.128356
5,I won't lose you the way I lost my mother. I am becoming more powerful than any Jedi has ever dreamed of. And I'm doing it for you. To protect you.,0.165211,0.000572,0.004698,0.008042,0.105926,0.799505,0.001522,0.002435,0.023411,0.006735,0.010634,0.001213,0.000761,0.005350,0.008621,0.016019,0.005381,0.007640,0.020575,0.006896,0.104707,0.008777,0.010219,0.011724,0.005370,0.023481,0.000978,0.061358
6,"Don't you see? We don't HAVE to run away anymore. I have brought PEACE to the Republic. I am more powerful than the Chancellor, I—I can overthrow him! And together, you and I can rule the galaxy, make things the way we want them to be!",0.233076,0.000819,0.001495,0.004269,0.486954,0.149432,0.003520,0.020883,0.038214,0.001884,0.003870,0.000378,0.000259,0.030626,0.001709,0.002263,0.000837,0.031590,0.008447,0.001854,0.233981,0.008857,0.011672,0.008154,0.000628,0.001606,0.001802,0.085752
7,I don't want to hear any more about Obi-Wan. The Jedi turned against me—don't YOU turn against me!,0.001119,0.001458,0.336668,0.365505,0.014059,0.005068,0.004058,0.002229,0.001700,0.012417,0.341223,0.011912,0.003004,0.001197,0.001565,0.001235,0.000497,0.001463,0.000711,0.000616,0.002435,0.000474,0.008487,0.000489,0.000658,0.003550,0.001381,0.225828
8,Because of Obi-Wan?,0.001454,0.001482,0.001687,0.007237,0.008366,0.000918,0.331714,0.326455,0.000723,0.001571,0.003576,0.001054,0.000638,0.001336,0.000766,0.000845,0.000242,0.000680,0.001210,0.000335,0.002031,0.000062,0.006632,0.000139,0.000352,0.000894,0.005571,0.611584
9,LIAR!,0.006682,0.003397,0.003256,0.005661,0.018140,0.000999,0.001475,0.001275,0.000924,0.002234,0.001545,0.003542,0.001161,0.005865,0.002786,0.000998,0.000560,0.003166,0.002484,0.000476,0.001399,0.000796,0.006646,0.000506,0.000417,0.002013,0.002115,0.944372
